In [1]:
%load_ext autoreload
%autoreload 2

In [182]:
import pandas as pd
from pykakasi import kakasi
from fugashi import Tagger
from jamdict import Jamdict

In [183]:
text = """
ポツダムにおける宣言
日本の降伏のための定義および規約
我々合衆国大統領、中華民国政府主席、及び英国総理大臣は、我々の数億の国民を代表し協議の上、日本国に対し戦争を終結する機会を与えることで一致した。
3ヶ国の軍隊は増強を受け、日本に最後の打撃を加える用意を既に整えた。この軍事力は、日本国の抵抗が止まるまで、同国に対する戦争を遂行する一切の連合国の決意により支持され且つ鼓舞される。
世界の自由な人民に支持されたこの軍事力行使は、ナチス・ドイツに対して適用された場合にドイツとドイツ軍に完全に破壊をもたらしたことが示すように、日本と日本軍が完全に壊滅することを意味する。
日本が、無分別な打算により自国を滅亡の淵に追い詰めた軍国主義者の指導を引き続き受けるか、それとも理性の道を歩むかを選ぶべき時が到来したのだ。
我々の条件は以下の条文で示すとおりであり、これについては譲歩せず、我々がここから外れることも又ない。執行の遅れは認めない。
日本国民を欺いて世界征服に乗り出す過ちを犯させた勢力を永久に除去する。無責任な軍国主義が世界から駆逐されるまでは、平和と安全と正義の新秩序も現れ得ないからである。
第6条の新秩序が確立され、戦争能力が失われたことが確認される時までは、我々の指示する基本的目的の達成を確保するため、日本国領域内の諸地点は占領されるべきものとする。
カイロ宣言の条項は履行されるべきであり、又日本国の主権は本州、北海道、九州及び四国ならびに我々の決定する諸小島に限られなければならない。
日本軍は武装解除された後、各自の家庭に帰り平和・生産的に生活出来る機会を与えられる。
我々の意志は日本人を民族として奴隷化し、また日本国民を滅亡させようとするものではないが、日本における捕虜虐待を含む一切の戦争犯罪人は処罰されるべきである。日本政府は日本国国民における民主主義的傾向の復活を強化し、これを妨げるあらゆる障碍は排除するべきであり、言論、宗教及び思想の自由並びに基本的人権の尊重は確立されるべきである。
日本は経済復興し、課された賠償の義務を履行するための生産手段、戦争と再軍備に関わらないものが保有出来る。また将来的には国際貿易に復帰が許可される。
日本国国民が自由に表明した意志による平和的傾向の責任ある政府の樹立を求める。この項目並びにすでに記載した条件が達成された場合に占領軍は撤退するべきである。
我々は日本政府が全日本軍の即時無条件降伏を宣言し、またその行動について日本政府が十分に保障することを求める。これ以外の選択肢は迅速且つ完全なる壊滅があるのみである。
""".strip()

In [184]:
tagger = Tagger()
jmd = Jamdict()
kks = kakasi()
kks.setMode("K", "H")
conv_hk = kks.getConverter()

In [17]:
data = pd.concat([pd.DataFrame.from_dict({"word": str(token), **token.feature._asdict()}, orient="index").T 
                  for token in tagger(text)]).reset_index(drop=True)

attr = ["orth", "lemma", "pos1", "pos2", "kana"]
dict_table = data[data["pos1"].isin("名詞、代名詞、形状詞、形容詞、連体詞".split("、"))][attr].drop_duplicates(subset="orth").reset_index(drop=True)
dict_table

,orth,lemma,pos1,pos2,kana
0,ポツダム,ポツダム-Potsdam,名詞,固有名詞,ポツダム
1,宣言,宣言,名詞,普通名詞,センゲン
2,日本,日本,名詞,固有名詞,ニッポン
3,降伏,降伏,名詞,普通名詞,コウフク
4,ため,為,名詞,普通名詞,タメ
...,...,...,...,...,...
164,十分,十分,形状詞,一般,ジュウブン
165,保障,保障,名詞,普通名詞,ホショウ
166,以外,以外,名詞,普通名詞,イガイ
167,選択,選択,名詞,普通名詞,センタク


In [109]:
result = jmd.lookup(dict_table.iloc[0]["orth"])

In [110]:
result.to_json()

{'entries': [],
 'chars': [],
 'names': [{'idseq': 5078797,
   'kanji': [],
   'kana': [{'text': 'ポツダム', 'nokanji': 0}],
   'senses': [{'SenseGloss': [{'lang': 'eng', 'text': 'Potsdam'}],
     'name_type': ['place']}]}]}

In [111]:
dir(result.entries[0].senses[0].gloss)
result.entries[0].senses[0].gloss

IndexError: list index out of range

In [115]:
result.names[0].senses[0].gloss

[Potsdam]

In [201]:
def display_result(result):
    if result.entries:
        entry = result.entries[0]
    elif result.names:
        entry = result.names[0]
    else:
        return "% MISSING"
    
    if entry.kanji_forms:
        kanji_forms = entry.kanji_forms
        kana_forms = entry.kana_forms
    else:
        kanji_forms = entry.kana_forms
        kana_forms = []        
    
    kanji = str.join("・", [kanji_form.text for kanji_form in kanji_forms])
    kana = str.join("・", [kana_form.text for kana_form in kana_forms])
    meaning = str.join("\n",[r"\item\justifying " + str.join("; ", [gloss.text for gloss in sense.gloss]) for sense in entry.senses])
    pos = ""
    
    return rf"\dictentry{{{kanji}}}{{{kana}}}{{{meaning}}}{{{pos}}}"

In [203]:
with open("tekstovi/potsdam_dict.tex", "w") as file:
    print(*[display_result(jmd.lookup(term)) for term in dict_table["orth"] if term], sep="\n", file=file)

In [187]:
entry = jmd.lookup("付ける").entries[0]
print(entry.text())

つける (付ける) : 1. to attach/to join/to add/to append/to affix/to stick/to glue/to fasten/to sew on/to apply (ointment) 2. to furnish (a house with) 3. to wear/to put on 4. to keep a diary/to make an entry 5. to appraise/to set (a price) 6. to allot/to budget/to assign 7. to bring alongside 8. to place (under guard or doctor) 9. to follow/to shadow 10. to load/to give (courage to) 11. to keep (an eye on) 12. to establish (relations or understanding) 13. to turn on (light) 14. to produce flowers/to produce fruit


In [194]:
dir(entry)
entry.senses[0].pos

['Ichidan verb', 'transitive verb']